In [1]:
from resnet_34_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        if m.bias is not None:  # Check if the bias exists
            torch.nn.init.normal_(m.bias.data)  # Xavier not applicable for biases
        
#TODO Get class weights
def getClassWeights():
    # TODO for Q4.c || Caculate the weights for the classes
    raise NotImplementedError

In [2]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20

n_class = 21

fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)

C:\Users\hejin\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hejin\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FCN_ResNet34(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-2)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)

In [4]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    best_iou_score = 0.0

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            # TODO / DONE  reset optimizer gradients
            
            optimizer.zero_grad()

            # both inputs and labels have to reside in the same device as the model's
            inputs =  inputs.to(device)# TODO / DONE transfer the input to the same device as the model's
            labels =  labels.to(device) # TODO / DONE transfer the labels to the same device as the model's

            outputs = fcn_model(inputs) # TODO / DONE Compute outputs. we will not need to transfer the output, it will be automatically in the same device as the model's!

            loss = criterion(outputs, labels) #TODO / DONE calculate loss

            loss.backward() # TODO / DONE backpropagate

            optimizer.step() # TODO / DONE update the weights


            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))

        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            # save the best model
    

In [5]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks

            # Take advantage of cuda if possible
#             if device == "cuda":
#                 input = input.cuda()

            input =  input.to(device)
            label = label.to(device)
    
            # Perform forward pass to get outputs.
            output = fcn_model.forward(input)
            N, numClass, H, W = output.shape

            # Find the prediction for each pixel
            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W)

            # Compute loss and accumulate it.
            
            loss = criterion(prediction.float(), label.float())
            losses.append(loss.item())
            
            # Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
            meanIOU = util.iou(prediction, label, n_class)
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label)
            accuracy.append(acc)

            
    print(f"Loss at epoch: {epoch} is {np.mean(losses)}")
    print(f"IoU at epoch: {epoch} is {np.mean(mean_iou_scores)}")
    print(f"Pixel acc at epoch: {epoch} is {np.mean(accuracy)}")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

#  #TODO
def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    # Asssume model loaded with the best weights.

    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):
            # Take advantage of cuda if possible
#             if device == "cuda":
#                 input = input.cuda()

            input =  input.to(device)
            label = label.to(device)

            # Perform forward pass to get outputs.
            output = fcn_model.forward(input)
            N, numClass, H, W = output.shape

            # Find the prediction for each pixel
            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W)

            # Compute loss and accumulate it.
            loss = criterion(prediction.float(), label.float())
            losses.append(loss.item())
            
            # Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
            meanIOU = util.iou(prediction, label, n_class)
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label)
            accuracy.append(acc)

    print(f"Loss at Test: {np.mean(losses)}")
    print(f"IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!



In [6]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [7]:
train()

epoch0, iter0, loss: 4.26560640335083
epoch0, iter10, loss: 2.5729687213897705
Finish epoch 0, time elapsed 48.69378876686096
Loss at epoch: 0 is 7847.386788504465
IoU at epoch: 0 is 0.038602838880159265
Pixel acc at epoch: 0 is 0.6315996620467382
epoch1, iter0, loss: 1.9677155017852783
epoch1, iter10, loss: 1.6702096462249756
Finish epoch 1, time elapsed 51.60322284698486
Loss at epoch: 1 is 3945.5577218191966
IoU at epoch: 1 is 0.04778093421262747
Pixel acc at epoch: 1 is 0.7472494675883745
epoch2, iter0, loss: 1.051594853401184
epoch2, iter10, loss: 1.6625539064407349
Finish epoch 2, time elapsed 66.89729309082031
Loss at epoch: 2 is 3668.1825823102677
IoU at epoch: 2 is 0.055647395717909796
Pixel acc at epoch: 2 is 0.7508111091813959
epoch3, iter0, loss: 1.2038421630859375
epoch3, iter10, loss: 1.2602026462554932
Finish epoch 3, time elapsed 62.61252808570862
Loss at epoch: 3 is 3663.6252790178573
IoU at epoch: 3 is 0.05564781345682834
Pixel acc at epoch: 3 is 0.7508232094456085
ep

In [8]:
modelTest()

Loss at Test: 4625.452183314732
IoU at Test: 0.056842249625016
Pixel acc at Test: 0.7280701395373999
